In [7]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.lines as mlines
import matplotlib.markers as mmarkers

matplotlib.rcParams['figure.dpi'] = 400

In [8]:
df = pd.read_csv("all_failures_database.csv")

In [9]:
grouped = df.groupby(["board", "family", "config", "bfm", "hardening"]).sum()
grouped.loc[pd.IndexSlice[:, :, :, "FLIP_SINGLE_BIT", :], "#faults"] = 750
grouped.loc[pd.IndexSlice[:, :, :, "RANDOM_VALUE", :], "#faults"] = 750
grouped.loc[pd.IndexSlice[:, :, :, "WARP_RANDOM_VALUE", :], "#faults"] = 250
grouped["PVF Critical SDC"] = grouped["critical_SDC"] / grouped["#faults"]
average_by_board = grouped["PVF Critical SDC"].reset_index().groupby(["board", "family", "bfm", "hardening"]).mean()
average_global = grouped["PVF Critical SDC"].reset_index().groupby(["family", "bfm", "hardening"]).mean()


In [10]:
grouped

SDC  critical_SDC  DUE  \
board  family config  bfm               hardening                            
Ampere EVA    B14-448 FLIP_SINGLE_BIT   Hardened     38             1   13   
                                        Unhardened   32             9   13   
                      RANDOM_VALUE      Hardened    313            13   27   
                                        Unhardened  185           138   21   
                      WARP_RANDOM_VALUE Hardened    195             7    0   
...                                                 ...           ...  ...   
Pascal Swin   L256    FLIP_SINGLE_BIT   Unhardened   59             1   19   
                      RANDOM_VALUE      Hardened    345             9   46   
                                        Unhardened  316            24   30   
                      WARP_RANDOM_VALUE Hardened    201            19    0   
                                        Unhardened  192            13    0   

                                                    #faults  PVF Critical SDC  
board  family config  bfm               hardening                              
Ampere EVA    B14-448 FLIP_SINGLE_BIT   Hardened      750.0          0.001333  
                                        Unhardened    750.0          0.012000  
                      RANDOM_VALUE      Hardened      750.0          0.017333  
                                        Unhardened    750.0          0.184000  
                      WARP_RANDOM_VALUE Hardened      250.0          0.028000  
...                                                     ...               ...  
Pascal Swin   L256    FLIP_SINGLE_BIT   Unhardened    750.0          0.001333  
                      RANDOM_VALUE      Hardened      750.0          0.012000  
                                        Unhardened    750.0          0.032000  
                      WARP_RANDOM_VALUE Hardened      250.0          0.076000  
                                        Unhardened    250.0          0.052000  

[144 rows x 5 columns]

In [11]:
average_by_board = average_by_board.unstack("hardening").reindex(["Original ViT", "EVA", "Swin", "MaxViT"], level="family").reindex(["FLIP_SINGLE_BIT", "RANDOM_VALUE", "WARP_RANDOM_VALUE"], level="bfm")
average_global = average_global.unstack("hardening").reindex(["Original ViT", "EVA", "Swin", "MaxViT"], level="family").reindex(["FLIP_SINGLE_BIT", "RANDOM_VALUE", "WARP_RANDOM_VALUE"], level="bfm")

In [12]:
with pd.ExcelWriter("~/git_research/date_2024_vits/data/bfmgraph_tmp.xlsx") as writer:
    average_by_board.to_excel(writer, sheet_name="byBoard")
    average_global.to_excel(writer, sheet_name="global")

In [13]:
raise NotImplementedError("Stop here")

NotImplementedError: Stop here

In [ ]:
# plot initialization

font_size = 14
col_order = ["Original", "EVA", "Swin", "MaxViT"]


In [ ]:
chart = sns.scatterplot(data=bfms, x="Family Model", y="Avg. Critical SDC", style="BFM", hue="Type", palette="flare", s=150,)

# avoid collision between points
for points in chart.collections:
    vertices = points.get_offsets().data
    if len(vertices) > 0:
        vertices[:, 0] += np.random.uniform(-0.3, 0.3, vertices.shape[0])
        points.set_offsets(vertices)

# setting axis labels
chart.set(xlabel=None)
chart.set_ylabel(ylabel="Average Critical SDC", fontsize=font_size)
plt.yticks(fontsize=font_size)
plt.xticks(fontsize=font_size)

# legend management
handles, labels = chart.get_legend_handles_labels()
handles_type = handles[1:3]
handles_bfm = handles[4:]

for hd in handles_type:
    hd.set_label("Hardened" if "H." == hd.get_label() else "Unhardened")

bfms_lgd = plt.legend(handles=handles_bfm, loc="upper center", bbox_to_anchor=(.50,1.11), ncols=len(handles_bfm), frameon=False,
                      fontsize=font_size, columnspacing=0.01, handletextpad=0.01)
for lg_hd in bfms_lgd.legendHandles:
    lg_hd._sizes = [100]
    
type_lgd = plt.legend(handles=handles_type, loc="upper center", bbox_to_anchor=(.50,1.18), ncols=len(handles_type), frameon=False,
           fontsize=font_size, columnspacing=0.01, handletextpad=0.01)
for lg_hd in type_lgd.legendHandles:
    lg_hd._sizes = [100]

plt.gca().add_artist(bfms_lgd)

plt.gca().yaxis.set_major_formatter(mtick.FuncFormatter(lambda y, _: f'{int(y)}%')) 
chart.set_xlim(-0.5, 3.5)

# plotting lines between the categories
plt.axvline(x=.5, color='black', ls=":", lw=1.75)
plt.axvline(x=1.5, color='black', ls=":", lw=1.75)
plt.axvline(x=2.5, color='black', ls=":", lw=1.75)
plt.tight_layout()
# saving the plot

In [ ]:
chart = sns.catplot(data=bfms, x="BFM", y="Avg. Critical SDC", hue="Type", col="Family Model", aspect=.4, col_order=col_order, legend=False)

# setting axis labels
chart.set(xlabel=None)
chart.set_titles(col_template='{col_name}')
chart.set_ylabels("Average Critical SDC", fontsize=font_size)
plt.xlabel("")
plt.yticks(fontsize=font_size)
plt.gca().yaxis.set_major_formatter(mtick.FuncFormatter(lambda y, _: f'{int(y)}%')) 
plt.xticks([])


plt.tight_layout()
plt.savefig("./avg_crit_sdc_catplot.pdf")

In [ ]:
chart = sns.stripplot(data=bfms,  x="Family Model", y="Avg. Critical SDC",  hue="Type", dodge=True, jitter=True)

# setting axis labels
chart.set(xlabel=None)

chart.set_ylabel("Average Critical SDC", fontsize=font_size)
plt.xlabel("")
plt.yticks(fontsize=font_size)
plt.gca().yaxis.set_major_formatter(mtick.FuncFormatter(lambda y, _: f'{int(y)}%')) 

plt.tight_layout()
plt.savefig("./avg_crit_sdc_catplot.pdf")